  # Comparing Groups

  This notebook gives an overview and introduction to comparing groups to understand if these groups are different from one another.. The data are real data collected from a group of young people during a Statistics class at FSEV UK. The data were retrieved from [Kaggle](https://www.kaggle.com/miroslavsabo/young-people-survey).

In [ ]:
# install packages for Google colab
install.pakcages(c("ggformula", "mosiac"))

# libraries
library(tidyverse)
library(ggformula)
library(mosaic)

# Set visualization theme
theme_set(theme_bw(base_size = 16))

# Read in Data
survey_data <- read_csv("https://raw.githubusercontent.com/lebebr01/jshs-stat/main/data/survey_responses.csv")

# Process data names for easier access
names(survey_data) <- gsub("-|,|/", "", names(survey_data))
names(survey_data) <- gsub("\\s+", "_", names(survey_data))

# Visual first few rows of the data
head(survey_data)


  ## Gender Differences in Phobias?

  Have you noticed if male or female individuals tend to have different phobias or more phobias? This example will explore if male or female individuals have greater levels of phobias in general. The survey data have information on a variety of phobias including:

 + flying
 + thunder, lightning
 + darkness
 + heights
 + spiders
 + snakes
 + rats, mice
 + ageing
 + dangerous dogs
 + public speaking

 

 All respondents were asked to respond whether they were not afraid at all (a score of 1 in the data) to very afraid of (a score of 5 in the data). These data are extracted below to show the first few individuals who responded to these 10 items.

In [ ]:
phobia_data <- survey_data %>%
   select(Flying, Storm, Darkness, Heights, Spiders, Snakes, Rats, Ageing, 
                Dangerous_dogs, Fear_of_public_speaking, Gender) %>%
                rowwise() %>%
   mutate(overall_phobia = sum(c_across(Flying:Fear_of_public_speaking))) %>%
   drop_na(Gender)

head(phobia_data)

 As you can see from the first few rows of data, there is variation in the individual responses to the 10 phobia questions. The code above also computed a new attribute representing the overall phobia level by adding all of the individual 10 survey questions together. Lower scores on this attribute represent individuals who do not have as many phobias and higher scores indicate higher or more phobias.

 ### Research Questions
 To articulate the questions we will explore in more detail

  1. Are there differences in the overall level of phobias based on the sex of the individual?   
  

 This could be translated into statistical hypotheses you may see in classical statistics books:

 $$ H_{0}: (\bar{X}_{male} - \bar{X}_{female}) = 0 $$
 $$ H_{1}: (\bar{X}_{male} - \bar{X}_{female})  \neq 0 $$

 The null hypothesis, $H_{0}$ represents no difference in the two groups. The alternative hypothesis represents that there is a difference in the mean of the two groups.

  Before we explicitly answer these research questions, let's first visualize the data to understand how many individuals in the data grew up in a city or rural location.

  ### How many males and females?

In [ ]:
gf_bar(~ Gender, data = phobia_data)


 ### Explore distribution of overall phobia.
 It is also helpful to explore the overall phobia attribute, which is serving as our outcome of interest or dependent variable.

In [ ]:
gf_histogram(~ overall_phobia, data = phobia_data, color = 'black') %>%
  gf_labs(x = "Overall Phobia (higher = more phobias)")


 ### Compare overall phobia across groups
 Once the distribution is explored as a whole, now it is helpful to see if the variation in the distribution can be explained by other attributes in our data. In this example, we are most interested in the sex of the individual.

In [ ]:
gf_violin(overall_phobia ~ Gender, data = phobia_data, fill = 'gray80', 
                 draw_quantiles = c(0.1, 0.5, 0.9)) %>%
        gf_labs(x = '', 
                        y = 'Overall Phobia (higher = more phobias)') %>%
        gf_refine(coord_flip())


  ## Fit a statistical model
 To more formally explore our research question, fitting a statistical model is helpful. This is helpful when we have a sample of data from a larger population we would wish to make a statement about. For example, in this situation, this may be a sample (of 1000 students) of a larger University student population. A statistical model can help us to quantify sampling error to determine if the differences we saw in the violin plot reflect signal or noise.

 To compare two groups, a linear model can be used to try to explain variation in the overall phobia based on group status. In order to do this, we must convert the text labels of the sex attribute to numeric quantities. For these data, we may do the following:

 + male = 1
 + female = 0  
 

 With code, we could do this as follows:

In [ ]:
phobia_data <- phobia_data %>%
    mutate(sex_numeric = ifelse(Gender == 'male', 1, 0))

count(phobia_data, Gender, sex_numeric)


 Notice from the output that male is associated with a 1 and female are associated with a 0. We can now use this numeric quantity in our linear model.

In [ ]:
phobia_model <- lm(overall_phobia ~ sex_numeric, data = phobia_data)

coef(phobia_model)


 Note, we could have also used the attribute, `Gender`, coded as is in the linear model.

In [ ]:
phobia_gender <- lm(overall_phobia ~ Gender, data = phobia_data)

coef(phobia_gender)


 To interpret these coefficients, it is helpful to compute the mean phobia level for males and females.

In [ ]:
phobia_data %>%
  df_stats(overall_phobia ~ Gender, mean)


 Notice from above, that the mean for the female group is the same value as the term for the intercept above. The intercept is interpreted as the mean when all of the attributes on the right hand side of the formula are equal to 0. In this case, the `Gender` and `sex_numeric` attributes are 0 for females. To understand what the other term is from the model above, we need to compute the difference in the computed means.

In [ ]:
22.13466 - 27.85592


 The term from the models labeled either "sex_numeric" or "Gendermale" is the same as the mean difference between male and female overall phobia. The model that was fitted earlier, is a linear regression, which looks like the following:

 $$ Y = mX + b + \epsilon $$

 where Y is the overall phobia, X is the Gender or sex_numeric attribute, b is the intercept, m is the linear slope (i.e., a mean difference here), and $\epsilon$ is an estimate of error.

 ## Estimating Error
 There are many ways to estimate error. In classical statistics, we would make some assumptions about the error in the data to analytically estimate the amount of error, largely based on the sample size and estimates of variation. This approach will be shown later, but first, let's explore an alternative that uses computational power to estimate the distribution of our effect of interest, the mean difference between the two groups.

 To do this, we use a procedure called the bootstrap. The bootstrap can be summarized into the following steps:
 1. Resample the original data, with replacement
 2. Fit the linear model
 3. Extract the coefficients
 4. Repeat steps 1 - 3 many times (i.e., 5,000, 10,000, or more times)
 5. Visualize distribution of mean differences

 Let's explore this in more detail by creating a function to do steps 1 - 3 one time.

In [ ]:
resample_phobia <- function(data) {
  resample_data <- data %>%
    sample_n(nrow(data), replace = TRUE)

   resample_data %>%
     lm(overall_phobia ~ Gender, data = .) %>%
     coef(.) %>%
     .[2] %>%
     data.frame()
}

resample_phobia(phobia_data)
resample_phobia(phobia_data)


 Let's now repeat this function many times.

In [ ]:
coef_phobia_diff <- map(1:10000, ~resample_phobia(data = phobia_data)) %>%
  bind_rows()
names(coef_phobia_diff) <- 'slope'

gf_density(~ slope, data = coef_phobia_diff) %>%
  gf_labs(x = "Mean Difference (male - female)")

gf_violin(1 ~ slope, data = coef_phobia_diff, fill = 'gray80', 
                 draw_quantiles = c(0.025, 0.5, 0.975)) %>%
   gf_labs(x = 'Mean Difference (male - female)',
                  y = '') %>%
    gf_refine(theme(axis.text.y = element_blank(),
                  axis.ticks.y = element_blank()))


 These quantiles can be computed manually to get the actual values. This can be compared to a confidence interval from the classical statistic approach.

In [ ]:
coef_phobia_diff %>%
  df_stats(~ slope, quantile(c(0.025, 0.5, 0.975)))

  confint(phobia_gender)
  